In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'ml-fundamentals-and-applications-2024-06:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F81723%2F8865197%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241014%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241014T152624Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9f2d7b2d8a16f7521ab4e576ffe7113bff257cbfcc1fe6370c3beacd1ca910dad9eb9d76412b575effdf8ee941a4030efc54f0b6158d68bd0deb9a91fcb687703532021595a5cd76726b708d3f1e821c2af118ab4386ec8f0918f1491a78bc6308f7d41a3c8f26e74d6ad06e463c8270fca742569f5269b32ce082acab1f622c18e705f8f91b340e6bf6cbed08e46412b65a5d5e83da634b4a4736ac4595209abc397f415ac1008bfede5ac00ee419fe223672e4f2539aa67150fe1e716943665c84878e19bb3c9d5f244ca48e6a1572f2ff348d0652d106f07515ae6e2d5070a582dc8fd00531af8829e933eff5f54db18acb338e36f65217bf7ef8e97d19f5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif

Link to competition: https://www.kaggle.com/competitions/ml-fundamentals-and-applications-2024-06


In [ ]:
data = pd.read_csv('/kaggle/input/ml-fundamentals-and-applications-2024-06/final_proj_data.csv')
test_data = pd.read_csv('/kaggle/input/ml-fundamentals-and-applications-2024-06/final_proj_test.csv')

In [ ]:
data = data.astype(str)
test_data = test_data.astype(str)

X = data.drop('y', axis=1)
y = data['y']

numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

numerical_transformer = imbpipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
    ('scaler', StandardScaler())
])

categorical_transformer = imbpipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

pipeline = imbpipeline(steps=[
    ('preprocessor', preprocessor),
    ('selectkbest', SelectKBest(score_func=f_classif)),
    ('smote', SMOTE(random_state=42)),
    ('classifier', GradientBoostingClassifier(random_state=42))
])

param_dist = {
    'selectkbest__k': [43],
    'smote__k_neighbors': [5],
    'classifier__n_estimators': [3],
    'classifier__learning_rate': [0.2],
    'classifier__max_depth': [5],
    'classifier__subsample': [0.78],
    'classifier__min_samples_split': [7]
}

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist,
                                   n_iter=1, scoring='balanced_accuracy',
                                   cv=5, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_pipeline = random_search.best_estimator_

y_val_pred = best_pipeline.predict(X_val)

balanced_accuracy_val = balanced_accuracy_score(y_val, y_val_pred)

print(f'Validation Balanced Accuracy: {balanced_accuracy_val}')

test_predictions = best_pipeline.predict(test_data)

test_predictions_df = pd.DataFrame({'index': test_data.index, 'y': test_predictions})

test_predictions_df.to_csv('submission.csv', index=False)